# 测试印尼语


In [3]:
file_path = "../.data/all_raretrible.txt"
# file_path = "../.data/mix_model/data/all_raretrible.txt"
# file_path = "/home/azhao/projects/ai/train/liubin/mix_model/data/all_raretrible.txt"
data = open(file_path, "r", encoding="utf-8").read().split("\n")
data = [x.strip().split(",") for x in data if len(x) > 0]
data = [[x[0], ",".join(x[1:]), "", "", "", ""] for x in data]
print(data[0:2])

[['感激', '好的,那我谢谢您了。', '', '', '', ''], ['中性', '没有词汇课的优惠券吗?', '', '', '', '']]


In [5]:
import pandas as pd

columns=["label", "text", "pred_label","main", "score", "keywords"]
# 转换 data 到 pandas DataFrame
df = pd.DataFrame(data, columns=columns)

# 保存为 CSV 文件
cvs_file = "../.data/data.csv"
df.to_csv(cvs_file, index=False)

In [6]:

from dspy.datasets import DataLoader

# 创建 dspy.datasets 对象
dl = DataLoader()
# fields 选择指定列 input_keys
all_dataset = dl.from_csv(
    cvs_file,
    fields=columns,
    input_keys=("text",),
)
print(all_dataset[0:2])

Generating train split: 0 examples [00:00, ? examples/s]

[Example({'label': '感激', 'text': '好的,那我谢谢您了。', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'}), Example({'label': '中性', 'text': '没有词汇课的优惠券吗?', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})]


In [7]:
from collections import defaultdict
import random


def stratified_sample(from_dataset, label_attr, *dataset_sizes):
    label_groups = defaultdict(list)
    for item in from_dataset:
        label = getattr(item, label_attr)
        label_groups[label].append(item)

    result_sets = [[] for _ in dataset_sizes]

    for label, items in label_groups.items():
        random.shuffle(items)
        label_sizes = [size // len(label_groups) for size in dataset_sizes]
        start = 0
        for i, size in enumerate(label_sizes):
            end = start + size
            result_sets[i].extend(items[start:end])
            start = end

    all_items = [item for items in label_groups.values() for item in items]
    for i, (result_set, target_size) in enumerate(zip(result_sets, dataset_sizes)):
        shortage = target_size - len(result_set)
        if shortage > 0:
            result_sets[i].extend(random.sample(all_items, shortage))

    return result_sets


In [8]:
# 使用示例
train_set, test_set, sample_set = stratified_sample(all_dataset, "label", 20, 50, 100)

In [9]:
from collections import Counter

train_set, test_set, sample_set = stratified_sample(all_dataset, "label", 20, 50, 100)
for dataset in [train_set, test_set, sample_set]:
    label_counts = Counter(item.get("label") for item in dataset)
    print(label_counts)

Counter({'中性': 5, '高兴': 4, '焦急': 3, '感激': 2, '惊讶': 2, '抱怨': 2, '生气': 2})
Counter({'抱怨': 8, '感激': 7, '中性': 7, '焦急': 7, '惊讶': 7, '高兴': 7, '生气': 7})
Counter({'感激': 16, '中性': 14, '焦急': 14, '惊讶': 14, '高兴': 14, '抱怨': 14, '生气': 14})


In [11]:
import random

random.shuffle(test_set)
for item in test_set:
    print(item)

Example({'label': '焦急', 'text': '好的,麻烦尽快,谢谢、', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})
Example({'label': '抱怨', 'text': '跑了几趟,感觉很麻烦,你直接帮我扣钱行了', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})
Example({'label': '焦急', 'text': '你们发票开的这么慢吗?', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})
Example({'label': '高兴', 'text': '太期待了', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})
Example({'label': '惊讶', 'text': '我昨天买的今天不是应该确认吗', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})
Example({'label': '感激', 'text': '好的,明白,谢谢你', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})
Example({'label': '惊讶', 'text': '只是协商嘛,不是退吗?你刚才不是说退吗?', 'pred_label': None, 'main': None, 'score': None, 'keywords': None}) (input_keys={'text'})
Example({'label': '中性', 'text

In [32]:
import importlib
import analysis

importlib.reload(analysis)

messages = []
for i, item in enumerate(test_set):
    messages.append(f"{i+1} {item.get('text')}")
ana = analysis.Analysis(messages=messages)
ret = ana.do()
print(ret.content)

d:\dev\py-venv\qa\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


You are a customer service quality inspection expert., sentiment analysis from Question

---

Follow the following format.

Question: list of input text
Answer: 
    返回格式 "序号,情绪分类,情绪子分类,情绪分类评分" 
    情绪分类包括 正向,负向,中性
    情绪子分类包括 ['中性', '惊讶', '感激', '抱怨', '焦急', '生气', '高兴']，
    情绪子分类评分(0-1.0)。
    例如 1,负向,抱怨,0.9
注意：请严格上面的分类来输出，不要输出不在情绪子分类包括中的的分类

Here is an example
---

Question: 好的谢谢!!
Answer: 1,正向,感激,0.8

---

Question: 谢了亲电话打过了
Answer: 2,正向,感激,0.8

---

Question: 我看下都吓我一跳
Answer: 3,负向,惊讶,0.9

---

Question: 哈哈哈哈可以的亲
Answer: 4,正向,高兴,0.8

---

Question:
```
0: 1 好的,麻烦尽快,谢谢、
1: 2 跑了几趟,感觉很麻烦,你直接帮我扣钱行了
2: 3 你们发票开的这么慢吗?
3: 4 太期待了
4: 5 我昨天买的今天不是应该确认吗
5: 6 好的,明白,谢谢你
6: 7 只是协商嘛,不是退吗?你刚才不是说退吗?
7: 8 我现在去你们的归还点都没有你们的机器
8: 9 好,谢谢你的耐心解答
9: 10 网速突然很卡
10: 11 辛苦了,谢谢你[emoji052]
11: 12 湖南能用???这个
12: 13 ??,谢谢
13: 14 至少要给我个时间点吧,催促了不止一次了
14: 15 非常好
15: 16 可以投诉你们对吧
16: 17 喔哈哈哈
17: 18 大额点的搞着都不敢用了
18: 19 我接的充电宝还不了
19: 20 不用抱歉!到时候跟警察抱歉去
20: 21 只是协商嘛,不是退吗?你刚才不是说退吗?
21: 22 好萌,哈哈
22: 23 已经造成很大麻烦!现在关掉吧
23: 24 他好像机子断

In [33]:
# 答案和问题对应起来
def update_sample(sample_set, answer):
    # 删除 answer 中的 'Answer: ', 删除空行
    answer = answer.replace("Answer: ", "").strip()
    for sample, prediction in zip(sample_set, answer.split("\n")):
        sample["score"] = prediction.split(",")[3]
        sample["pred_label"] = prediction.split(",")[2]
        sample["main"] = prediction.split(",")[1]


def sample_error_rate(sample_set):
    incorrect_count = sum(
        1 for sample in sample_set if sample["pred_label"] != sample["label"]
    )
    total_count = len(sample_set)
    return incorrect_count / total_count

In [34]:
update_sample(test_set, ret.content)
print(sample_error_rate(test_set))


0.78


In [35]:
idx = 0
for sample in test_set:
    if sample["pred_label"] != sample["label"]:
        print(f"Question {idx}: {sample}")
        idx+=1

Question 0: Example({'label': '焦急', 'text': '好的,麻烦尽快,谢谢、', 'pred_label': '感激', 'main': '正向', 'score': '0.8', 'keywords': '正向'}) (input_keys={'text'})
Question 1: Example({'label': '焦急', 'text': '你们发票开的这么慢吗?', 'pred_label': '抱怨', 'main': '负向', 'score': '0.8', 'keywords': '负向'}) (input_keys={'text'})
Question 2: Example({'label': '高兴', 'text': '太期待了', 'pred_label': '期待', 'main': '正向', 'score': '0.8', 'keywords': '正向'}) (input_keys={'text'})
Question 3: Example({'label': '惊讶', 'text': '我昨天买的今天不是应该确认吗', 'pred_label': '催促', 'main': '中性', 'score': '0.7', 'keywords': '负向'}) (input_keys={'text'})
Question 4: Example({'label': '感激', 'text': '好的,明白,谢谢你', 'pred_label': '感谢', 'main': '中性', 'score': '0.8', 'keywords': '正向'}) (input_keys={'text'})
Question 5: Example({'label': '惊讶', 'text': '只是协商嘛,不是退吗?你刚才不是说退吗?', 'pred_label': '疑问', 'main': '中性', 'score': '0.7', 'keywords': '负向'}) (input_keys={'text'})
Question 6: Example({'label': '中性', 'text': '我现在去你们的归还点都没有你们的机器', 'pred_label': '抱怨', 'main': '负向